<a href="https://colab.research.google.com/github/AndyAlyf/Klasifikasi-Text/blob/master/classifierProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import csv
import math
import os
from collections import OrderedDict
import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

class preProcessing():
    def parsing(dokumen):
        stopword = open("stopword.txt","r").read().split("\n")
        dokumen = dokumen.lower().split()
        dokumen = [kata for kata in dokumen if kata not in stopword]
        # print(dokumen)
        return dokumen
    
    def cleaning(dokumen):
        tandabaca = string.punctuation
        domain = re.compile(r'[\S]+\.(net|com|org)[\S]*\s?')
        dokumenproses = dokumen.strip()
        dokumenproses = domain.sub('',dokumenproses)
        hasil = ''
        for i,karakter in enumerate(dokumenproses):
            if (karakter in tandabaca or karakter.isnumeric()):
                if (i != len(dokumenproses)-1):
                    nextkarakter = dokumenproses[i+1]
                    if (nextkarakter not in tandabaca and not nextkarakter.isnumeric() and nextkarakter != ' '):
                        hasil += ' '
            else:
                hasil += karakter
    #     print(hasil)
        return hasil

    # def type(dokumen):
    #     return list(OrderedDict(kata,None)for kata in dokumen).keys()

    def stemming(dokumen):
    #     print([createstr.stem(kata) for kata in dokumen if kata and kata.isascii()])
        return [StemmerFactory().create_stemmer().stem(kata) for kata in dokumen if kata and kata.isascii()]

    def preprocessing(dokumen):
        return stemming(
            parsing(
                cleaning(dokumen)
            )
        )

    def main():
        direktori = input("masukkan direktori yang ingin dibaca: ")
        if(direktori.lower().endswith(".txt")):
            file = open(direktori,"r",encoding="ISO-8859-1")
            konten = file.read()
            file.close()
            hasil = preprocessing(konten)
            print(hasil)

class naifeBayes():
    def __init__(self,data,kolom):
        self.data=data
        self.kolom=kolom
        self.label = []
        for i in range(len(self.data)):
            if(self.data['LABEL'][i]) not in self.label:
                self.label+=[self.data['LABEL'][i]]

    def prints(self):
        print(self.data,self.kolom,self.label)

    def prediksi(self,pred1,predk):
        high = 0
        for i in self.label:
            hasil = self.pcd(i,pred1,predk)
            if hasil>high:
                high = hasil
                kats = i
        return kats

    def pcd(self,c,pred1,predk):
        temp = 1
        for i in predk:
            temps = 1
            if(pred1[i]>1):
                temps = math.pow(self.pdc(i,c),pred1[i])
                temp = temp*temps
            else:
                temp = temp*self.pdc(i,c)
        temp = temp*self.pc(c)
        return temp

    def pc(self,c):
        c = str(c)
        count = 0
        leng = len(data)
        for i in range(leng):
            if (data['LABEL'][i]==c):
                count+=1
        temp = count/leng
        return temp

    def pdc(self,w,c):
        countv = len(kolom)
        sumw = 0
        countwc = 0
        for i in range(len(data)):
            sumw+=data[w][i]
            if(data['LABEL'][i]==c):
                countwc+=data[w][i]
        atas = countwc + 1
        bawah = sumw + countv
        temp = atas/bawah
        return temp
    

data = pd.read_csv("Testes.csv")
# ambil kolom sebagai list dengan nama kolom
kolom=[]
with open("Testes.csv") as csf:
    readCSV = csv.reader(csf,delimiter=",")
    for i in readCSV:
        kolom += i
        kolom = kolom[:-1]
        break

# untuk diprediksi nanti
# memetakan kata unik / kolom yang ada
l1st=[0,0,0,0,0,0,3,2,1,0,1,0,0]
pred = dict()
pred1 = dict()
predk=[]
j=0
for i in kolom:
    pred[i] = l1st[j]
    if pred[i]>0:
        pred1[i] = l1st[j]
        predk+=[i]
    j+=1
    
print(data)
print(pred)
print(pred1)
print(predk)
print()
c = naifeBayes(data,kolom)
print(c.label)
high= 0
kats =''

for i in c.label:
    hasil = c.pcd(i,pred1,predk)
    print("Peluang masuk kategori",i,"adalah :",hasil)

print()
print("Jadi, document ini termasuk pada kategori :",c.prediksi(pred1,predk))

   suka  masak  nasi  goreng  ukur  makan  beras  tanam  sawah  mobil  bus  \
0     2      3     3       3     0      0      0      0      0      0    0   
1     0      0     2       0     1      1      0      0      0      0    0   
2     0      0     1       0     0      0      2      2      2      0    0   
3     0      0     0       0     1      0      0      0      0      4    2   
4     0      0     0       0     1      0      0      0      1      0    2   
5     1      2     4       2     1      1      1      0      0      0    0   
6     1      3     2       1     1      2      0      0      0      0    0   
7     3      0     0       0     2      0      0      0      0      0    1   
8     1      0     3       0     2      3      1      2      3      0    0   

   angkut  tumpang     LABEL  
0       0        0   kuliner  
1       0        0   kuliner  
2       0        0   ekonomi  
3       1        1  otomotif  
4       0        0  otomotif  
5       0        0   kuliner  
6 